In [0]:
!cat pyproject.toml

[tool.poetry]
name = "ehr-relation-extraction"
version = "0.1.0"
description = ""
authors = ["Ken Dwyer <kdd133@gmail.com>"]
readme = "README.md"
packages = [{include = "ehr_relation_extraction"}]

[tool.poetry.dependencies]
python = "3.10.12"
scispacy = "0.5.3"
torch = "1.13.1"
transformers = "4.30.2"
numpy = "1.21.5"
seqeval = "^1.2.2"
pandas = "1.4.4"
filelock = "^3.13.1"
scikit-learn = "1.1.1"
networkx = "^3.2.1"
matplotlib = "^3.8.2"
en-ner-bc5cdr-md = {url = "https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_ner_bc5cdr_md-0.5.3.tar.gz"}
ipython = "^8.19.0"
accelerate = "0.20.3"


[build-system]
requires = ["poetry-core"]
build-backend = "poetry.core.masonry.api"


In [0]:
# The package versions match the ones in the Databricks 13.3 ML runtime where possible.
# We only need to install the packages that aren't already in the runtime.
%pip install --upgrade pip
%pip install "seqeval==1.2.2" "filelock==3.13.1" "https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_ner_bc5cdr_md-0.5.3.tar.gz"

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.2.2
    Uninstalling pip-22.2.2:
      Successfully uninstalled pip-22.2.2
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 21.8 MB/s eta 0:00:00
  Created wheel for en-ner-bc5cdr-md: filename=en_ner_bc5cdr_md-0.5.3-py3-none-any.whl size=119790315 sha256=49a946d63c9d7c26a9bf53c6a21a85493b3be5f682dfd790f8ae9763a60bf210
  Stored in directory: /root/.ca

In [0]:
ls /databricks/driver

azure/  eventlogs/                   logs/
conf/   hadoop_accessed_config.lst*  preload_class.lst*


In [0]:
rm -r /databricks/driver/datasets/preprocessed /databricks/driver/output/ner_sample

rm: cannot remove '/databricks/driver/output/ner_sample': No such file or directory


In [0]:
mkdir -p /databricks/driver/datasets/ner_sample/train /databricks/driver/datasets/ner_sample/test /databricks/driver/datasets/preprocessed

In [0]:
cp -v /dbfs/FileStore/n2c2-2018/training_20180910/107*.* /databricks/driver/datasets/ner_sample/train/

'/dbfs/FileStore/n2c2-2018/training_20180910/107047.ann' -> '/databricks/driver/datasets/ner_sample/train/107047.ann'
'/dbfs/FileStore/n2c2-2018/training_20180910/107047.txt' -> '/databricks/driver/datasets/ner_sample/train/107047.txt'
'/dbfs/FileStore/n2c2-2018/training_20180910/107058.ann' -> '/databricks/driver/datasets/ner_sample/train/107058.ann'
'/dbfs/FileStore/n2c2-2018/training_20180910/107058.txt' -> '/databricks/driver/datasets/ner_sample/train/107058.txt'
'/dbfs/FileStore/n2c2-2018/training_20180910/107139.ann' -> '/databricks/driver/datasets/ner_sample/train/107139.ann'
'/dbfs/FileStore/n2c2-2018/training_20180910/107139.txt' -> '/databricks/driver/datasets/ner_sample/train/107139.txt'
'/dbfs/FileStore/n2c2-2018/training_20180910/107251.ann' -> '/databricks/driver/datasets/ner_sample/train/107251.ann'
'/dbfs/FileStore/n2c2-2018/training_20180910/107251.txt' -> '/databricks/driver/datasets/ner_sample/train/107251.txt'
'/dbfs/FileStore/n2c2-2018/training_20180910/107255.ann'

In [0]:
cp -v /dbfs/FileStore/n2c2-2018/training_20180910/107*.* /databricks/driver/datasets/ner_sample/test/

'/dbfs/FileStore/n2c2-2018/training_20180910/107047.ann' -> '/databricks/driver/datasets/ner_sample/test/107047.ann'
'/dbfs/FileStore/n2c2-2018/training_20180910/107047.txt' -> '/databricks/driver/datasets/ner_sample/test/107047.txt'
'/dbfs/FileStore/n2c2-2018/training_20180910/107058.ann' -> '/databricks/driver/datasets/ner_sample/test/107058.ann'
'/dbfs/FileStore/n2c2-2018/training_20180910/107058.txt' -> '/databricks/driver/datasets/ner_sample/test/107058.txt'
'/dbfs/FileStore/n2c2-2018/training_20180910/107139.ann' -> '/databricks/driver/datasets/ner_sample/test/107139.ann'
'/dbfs/FileStore/n2c2-2018/training_20180910/107139.txt' -> '/databricks/driver/datasets/ner_sample/test/107139.txt'
'/dbfs/FileStore/n2c2-2018/training_20180910/107251.ann' -> '/databricks/driver/datasets/ner_sample/test/107251.ann'
'/dbfs/FileStore/n2c2-2018/training_20180910/107251.txt' -> '/databricks/driver/datasets/ner_sample/test/107251.txt'
'/dbfs/FileStore/n2c2-2018/training_20180910/107255.ann' -> '/da

In [0]:
!make generate-data

python generate_data.py \
--task ner \
--input_dir /databricks/driver/datasets/ner_sample \
--target_dir /databricks/driver/datasets/preprocessed/ner_sample \
--max_seq_len 512 \
--dev_split 0 \
--tokenizer biobert-base \
--ext txt \
--sep " "
2024-01-06 23:39:45.981459: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.

Reading data

Train data:
Progress: [====================] 8/8

Test Data:
Progress: [====================] 8/8

Data successfully saved in /databricks/driver/datasets/preprocessed/ner_sample/train.txt
Variable successfully saved in /databricks/driver/datasets/preprocessed/ner_sample/train.pkl
Data successfully saved in /databricks/driver/datasets/preprocessed/ner_sample/train_dev.txt
Variable successfully sav

In [0]:
!make train-biobert-ner

cd biobert_ner/ && \
python run_ner.py \
    --data_dir /databricks/driver/datasets/preprocessed/ner_sample/ \
    --labels /databricks/driver/datasets/preprocessed/ner_sample/labels.txt \
    --model_name_or_path dmis-lab/biobert-large-cased-v1.1 \
    --output_dir /databricks/driver/output/ner_sample/ \
    --max_seq_length 512 \
    --num_train_epochs 10 \
    --per_device_train_batch_size 4 \
    --save_steps 20 \
    --seed 0 \
    --do_train \
    --do_eval \
    --do_predict \
    --report_to "none"
2024-01-06 23:40:19.421855: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
01/06/2024 23:40:21 - WARNING - __main__ -   Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: True, 16-bits training: False
01/06/